In [ ]:
# 논문 : neural collaborative filtering

In [ ]:
몇년간 speech recognition, computer vision, nlp 분야에서는 딥러닝에 대한 연구가 활발했지만
추천시스템에서는 딱히 딥러닝이 활용되지 않았었음
모델의 input으로 들어가는 item정보등에 딥러닝을 활용했지만 user-item interaction은 MF나 inner product를 씀..

In [ ]:
이 논문에서는 user - item interaction을 학습하는데 multi-layer perceptron을 활용함..
matrix factorization은 mlp의 특수한 case중 하나이며, mlp로 mf를 일반화하고 표현할수있음

In [ ]:
개인화 추천의 key는 과거 user-item interaction에 기반하여 user의 item에대한 선호도를 모델링하는것..
mf는 가장 유명한것이고, user와 item을 공유되는 latent space에 투영시키는것..

mf의 효율성에도 불구하고 mf의 interaction function(inner product)때문에 mf의 performance는 방해받는다..
(inner product가 아닌 것을 사용할 수 있다면 더좋은 performance를 낼 수 있다는 뜻으로 이해함)

-> inner product는 단순히 latent feature들의 곱을 linear하게 합친거기때문에 user-item interaction의 복잡성을
설명하기에는 다소 부족함이 있다고 설명하였음

최근 발전으로 DNN이 item의 정보를 모델링하는데 활용되었음
-> textual description of items, audio features of music 등등..

이 논문에서는 implicit feedback에 집중하였는데.. 영상을 보거나 상품을 사거나 아이템을 클릭하는등의 행위

explicit feedback과 달리 implicit feedback은 자동적으로 추적가능하고 훨씬 수집하기가 쉬움
하지만 활용하기는 어려움
-> user의 만족도에 대한 내용이 없고 negative feedback이 부족하기 때문

In [ ]:
main contributions

1. nn architecture를 user와 item의 latent feature를 모델링하는데 활용하고 NCF(Neural Collaborative Filtering)
제안

2. MF는 NCF의 특별한 케이스로 해석가능함을 보이고 NCF모델링에 high-level non-linearity를 부여하기위해 활용

3. 2개의 real-world 데이터에 적용해보고 ncf의 효율성을 선보임

In [ ]:
user-item matrix Y를 정의하는데 y_ui가 1이면 interaction이 있는경우, 0이면 없는경우..

1이라고 해서 item을 선호하는것은 아니고 0이라고해서 비선호가 아니다..
interaction 그 자체의 유무를 나타내는것..

<img src = "./1.png">

In [ ]:
u4 와 나머지의 유사도를 구하면.. u1, u3, u2 순.. 그런데 p2가 p3보다 더 유사하게 나오는데 이것은
inner product가 linear하기 때문에 발생하는 문제

latent space의 dimension을 높여서 해결할 수 있지만 그럴경우 generalization에 문제가 생길수있음
(overfitting이 발생하는 등..)

<img src = "./2.png">

In [ ]:
ncf구조..
user embedding과 item embedding을 latent space로 볼 수 있다.

실제 y_ui값과 에측 y_ui값 사이의 pointwise loss를 줄이는 학습이 이루어진다..

squared loss는 gaussian distribution으로부터 observation들이 나온다고 가정할때 설명이 되는데..
이것은 implicit data에 잘 해당되지 않는 이야기라고 한다..
그래서 probabilistic approach를 하게되는데..이를 위해서는 예측 y_ui의 값을 [0, 1]로 제한해야한다..

이런 가정하에 likelihood function을 구해서 식을 정리해보면
binary cross-entropy loss와 동일한 형태의 식이 나오게 된다..
학습은 SGD로..

<img src = "./3.png">

In [ ]:
p를 user latent vector, q를 item latent vector라고 했을때..
activation function a_out을 identity function, weight h를 1이라고 하면.. mf model이 된다.

여기에서 h가 uniform 제한 없이 학습되게끔 한다면 latent dimension의 중요성이 변화하는 mf를 만들것이고..
a_out을 non-linear한 function을 사용하면.. linear mf model보다 더 표현력있는 mf가 될것이다...

여기서는 sigmoid를 사용한다..

In [ ]:
ncf에서는 user와 item을 모델하기위해 두가지 경로르 채택한다.. 이때 그둘을 concat함..
단순 concat은 user와 item사이의 interaction을 표현하기에는 부족하므로..
mlp를 활용하여 interaction을 학습한다

각 mlp layer마다 activation function은 여러가지중에 자유롭게 선택할수있는데 각각 장단점이 있음
여기서는 ReLU를 활용함
그리고 mlp는 tower pattern을 따르는데 (bottom layer가 가장 넓고 top layer가 가장 좁은..)
higher layer에서 더 적은 hidden unit을 이용하는건.. 이렇게 해야 데이터의 추상적인..abstrative한 feature를
학습할 수 있기 때문이라고 한다.

<img src = "./4.png">

In [ ]:
gmf와 mlp를 합치기위해 위와 같은 구조를 사용하는데
gmf와 mlp사이에 weight를 이용해서 trade-off를 조절 가능하다..
그리고 gmf와 mlp의 embedding layer도 각각 사용한다..
같은 것을 사용할경우 fused model의 performance를 제한시킨다..

각각 학습하고 마지막 hidden layer에서 두개를 concat함

gmp와 mlp는 adam으로 pretrain하고 합친후에 NeuMF를 vanilla sgd로 학습시킨다..

In [ ]:
# https://github.com/hexiangnan/neural_collaborative_filtering
위 github에 구현된 코드가있는데 직접구현도 해보고 공부도 해봐야할듯..